In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# if you have made changes to a local copy of astroplan you have changed use:
import sys
#sys.path.insert(0,'directorypath')
sys.path.insert(0,'/home/karl/Astroplan/astroplan/')

In [3]:
# import the parts of astropy and astroplan needed
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
import astroplan
import astroplan.constraints
from astroplan import Observer, FixedTarget, ObservingBlock
from astroplan import PriorityScheduler, Transitioner
from astroplan import plots
color_cycle = plots.mplstyles.astropy_mpl_style['axes.color_cycle']

In [4]:
# define the start and end time of the schedule
start_time = Time.now()
end_time = start_time+48*u.hour
print(start_time,end_time)

2016-07-28 05:12:20.757045 2016-07-30 05:12:20.757045


In [5]:
# define the observer (in this example we use mdm)
mdm = Observer.at_site('mdm')

In [6]:
# now we create a list of the targets so that we can iterate when making their blocks
targets = [FixedTarget.from_name('Vega'),
           FixedTarget.from_name('Deneb'),
           FixedTarget.from_name('Arcturus'),
           FixedTarget.from_name('Altair'),
           FixedTarget.from_name('Aldebaran'),
           FixedTarget.from_name('Sirius'),
           FixedTarget.from_name('Betelgeuse'),
           FixedTarget.from_name('Rigel'),
           FixedTarget.from_name('Castor'),
           FixedTarget.from_name('Pollux'),
           FixedTarget.from_name('Polaris')
          ]
targets

[<FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg (279.23473479, 38.78368896)>,
 <FixedTarget "Deneb" at SkyCoord (ICRS): (ra, dec) in deg (310.35797975, 45.28033881)>,
 <FixedTarget "Arcturus" at SkyCoord (ICRS): (ra, dec) in deg (213.9153003, 19.18240916)>,
 <FixedTarget "Altair" at SkyCoord (ICRS): (ra, dec) in deg (297.6958273, 8.8683212)>,
 <FixedTarget "Aldebaran" at SkyCoord (ICRS): (ra, dec) in deg (68.98016279, 16.50930235)>,
 <FixedTarget "Sirius" at SkyCoord (ICRS): (ra, dec) in deg (101.28715533, -16.71611586)>,
 <FixedTarget "Betelgeuse" at SkyCoord (ICRS): (ra, dec) in deg (88.79293899, 7.40706399)>,
 <FixedTarget "Rigel" at SkyCoord (ICRS): (ra, dec) in deg (78.63446707, -8.20163837)>,
 <FixedTarget "Castor" at SkyCoord (ICRS): (ra, dec) in deg (113.64947164, 31.88828222)>,
 <FixedTarget "Pollux" at SkyCoord (ICRS): (ra, dec) in deg (116.32895777, 28.02619889)>,
 <FixedTarget "Polaris" at SkyCoord (ICRS): (ra, dec) in deg (37.95456067, 89.26410897)>]

In [7]:
# an observing block requires a target, a duration, and a priority
# this uses .from_exposures to input exposure time, read-out time,
# and the number of exposures as the duration
etime = 5*u.min
n = 10
rot = 55*u.second
# Constraints can also be added to the ObservingBlock:
constraint = [astroplan.constraints.AtNightConstraint()]
# The scheduler needs a list of blocks:
blocks = []
for i,t in enumerate(targets):
    #i is used as the priority in this case
    blocks.append(ObservingBlock.from_exposures(t, i, etime, n, rot, constraints = constraint))

In [8]:
# constraints that apply to every ObservingBlock can also be given directly to the Scheduler
global_constraints = [astroplan.constraints.AirmassConstraint(3, boolean_constraint=False)]
# Soon, this should be able to handle a vector input (e.g. AirmassConstraint([3,3,3,3,2,2,3,3,3,2,2]))

# define how long it takes the telescope to transition (e.g. 1 degree/second)
trans = Transitioner(slew_rate=1*u.deg/u.second)

In [ ]:
# The PriorityScheduler schedules the block with priority closest to 0 first
# and proceeds according to the priority.
# create the scheduler object
scheduler = PriorityScheduler(start_time, end_time,transitioner=trans,
                              constraints=global_constraints, observer=mdm)
# run the scheduler, this scheduler will also print unschedulable targets
schedule = scheduler(blocks)
# to print a table of the output schedule (showing unused slots):
schedule.to_table(show_unused=True).show_in_notebook()

could not schedule Sirius
could not schedule Betelgeuse
could not schedule Rigel
could not schedule Castor
could not schedule Pollux


idx,target,start time (UTC),end time (UTC),duration (minutes),ra,dec
0,Unused Time,2016-07-28 05:12:20.757,2016-07-28 07:03:00.761,110.666734355,,
1,Deneb,2016-07-28 07:03:00.761,2016-07-28 08:03:00.761,60.0,310d21m28.7271s,45d16m49.2197s
2,TransitionBlock,2016-07-28 08:03:00.761,2016-07-28 08:04:00.761,1.0,,
3,Unused Time,2016-07-28 08:04:00.761,2016-07-28 11:24:00.771,200.000159457,,
4,Polaris,2016-07-28 11:24:00.771,2016-07-28 12:24:00.771,60.0,37d57m16.4184s,89d15m50.7923s
5,TransitionBlock,2016-07-28 12:24:00.771,2016-07-28 12:25:20.771,1.33333333333,,
6,Unused Time,2016-07-28 12:25:20.771,2016-07-29 02:21:40.804,836.333881761,,
7,Arcturus,2016-07-29 02:21:40.804,2016-07-29 03:21:40.804,60.0,213d54m55.0811s,19d10m56.673s
8,TransitionBlock,2016-07-29 03:21:40.804,2016-07-29 03:22:40.804,1.0,,
9,Unused Time,2016-07-29 03:22:40.804,2016-07-29 04:55:00.809,92.333427012,,


In [ ]:
# We can also plot the airmass of the targets compared to when they were scheduled
plt.figure(figsize=(15,10))
# show_night makes the graphs much slower, but gives useful information if
# "AtNight" was a constraint.
plots.plot_schedule_airmass(schedule, 
                            show_night=True
                            )